# Data Engineering - Capstone Project - ETL Pipeline

### Step 1: Scope the Project and Gather Data

This project aims to gather electrical consumption data from multiple sources, clean and aggregate this data into one consolidated single source of truth database ready to be queried by analysts.  Data will be aggregated into daily and monthly values and placed into seperate tables to speed up analytical queries.  
For the purpose of the project data will come from three seperate sources, these contain actual *anonmyised* submeter and main meter data which have been placed in an S3 bucket for learning purposes.  This is a proof of concept idea, to demonstrate knowledge of data engineering principles (AWS and Data warehouisng) learnt throughout the Nanodegree.
Each sources contains half-hourly kilowatt consumption values, with some of these dating back to 2015.  

A more detailed breakdown of these can be found below.

#### Source 1:
- Subset of 100 different submeter values dating back to 2015.
- Derived from Tridium JACE devices.
- Values are given as sequential meter reads.
- Values have been exported in CSV format and are located on an S3 bucket.

#### Source 2: 
- Subset of 200 different submeter values dating back to 2019.
- Values derived from a proprietary sub meter data provider.
- Values are given as delta between sequential meter reads, this is the actual amount consumed in that half-hourly period.
- Values have been exported to JSON format and are located on an S3 bucket.

#### Source 3: 
- Subset of monthly main meter reads for June 2018.
- Values are given as the actual amount consumed in that half-hourly period.
- Values have been exported to JSON format and are lcoated in an S3 bucket.



